# TSA Chapter 7: PPP Cointegration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch7/TSA_ch7_ppp_cointegration/TSA_ch7_ppp_cointegration.ipynb)

Purchasing Power Parity: USD/EUR exchange rate and US/EU CPI ratio from FRED, 2000-2024.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas_datareader yfinance -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, coint
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Fetch real data from FRED
start, end = '2000-01-01', '2024-12-31'

# USD/EUR exchange rate (daily -> resample to monthly)
exrate = web.DataReader('DEXUSEU', 'fred', start, end).dropna()
exrate_m = exrate.resample('MS').mean().dropna()

# US CPI (monthly, all urban consumers)
us_cpi = web.DataReader('CPIAUCSL', 'fred', start, end).dropna()

# EU CPI (monthly, Euro area HICP)
eu_cpi = web.DataReader('CP0000EZ19M086NEST', 'fred', start, end).dropna()

# Align all series on common monthly dates
data = pd.DataFrame({
    'USDEUR': exrate_m['DEXUSEU'],
    'US_CPI': us_cpi['CPIAUCSL'],
    'EU_CPI': eu_cpi['CP0000EZ19M086NEST']
}).dropna()

# Compute log exchange rate and log CPI ratio
log_exrate = np.log(data['USDEUR'])
log_cpi_ratio = np.log(data['US_CPI'] / data['EU_CPI'])
ppp_deviation = log_exrate - log_cpi_ratio

print(f'Period: {data.index[0].strftime("%Y-%m")} to {data.index[-1].strftime("%Y-%m")}, n={len(data)}')

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Panel 1: Log exchange rate vs log CPI ratio
axes[0].plot(data.index, log_exrate, color=BLUE, lw=1.2, label='Log USD/EUR Rate')
axes[0].plot(data.index, log_cpi_ratio, color=RED, lw=1.2, label='Log CPI Ratio (US/EU)')
axes[0].set_title('PPP: Exchange Rate vs CPI Ratio (FRED)', fontweight='bold')
axes[0].set_ylabel('Log scale')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)

# Panel 2: PPP deviation
axes[1].plot(data.index, ppp_deviation, color=GREEN, lw=1)
axes[1].axhline(ppp_deviation.mean(), color=GRAY, ls='--', lw=0.8, label=f'Mean = {ppp_deviation.mean():.3f}')
axes[1].set_title('PPP Deviation (Mean-Reverting)', fontweight='bold')
axes[1].set_ylabel('Deviation')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=1, frameon=False)

plt.tight_layout()
save_chart(fig, 'ppp_cointegration')
plt.show()

# Cointegration test
t_stat, p_val, _ = coint(log_exrate, log_cpi_ratio)
print(f'\nEngle-Granger cointegration test: t-stat={t_stat:.3f}, p-value={p_val:.4f}')
adf_dev = adfuller(ppp_deviation)
print(f'ADF on PPP deviation: t-stat={adf_dev[0]:.3f}, p-value={adf_dev[1]:.4f}')